# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# VERSIONS #
### GPU 0 : NVIDIA GeForce RTX 3080 Ti Laptop GPU
### GPU 1 : AMD Radeon(TM) Graphics
### CPU : AMD Ryzen 9 6900HX with Radeon Graphics

### -------------------------- Python & library version --------------------------
### Python version: 3.9.13 (main, Aug 25 2022, 23:51:50) [MSC v.1916 64 bit (AMD64)]
### pandas version: 2.0.3
### numpy version: 1.21.5
### matplotlib version: 3.5.2
### tqdm version: 4.64.1
### xgboost version: 1.7.2
### lightgbm version: 3.3.3
### catboost version: 1.1.1
### scikit-learn version: 1.0.2
------------------------------------------------------------------------------

In [81]:
# <!-- from google.colab import drive
# drive.mount('/content/drive')# Import library -->
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import xgboost as xgb
from tqdm import tqdm  # tqdm 임포트
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
seed_everything(42) # Seed 고정

In [82]:
# Read ML Predict Proba
LGBM = pd.read_csv('LGBM_FINAL_PROBA.csv')


In [83]:
# Read DL Predict Proba (Xtransformer and Saint sub)

xtf = pd.read_csv('final_xtf_proba.csv')

submission_saint_1 = pd.read_csv('saint_transformer_predict_proba_real_final.csv')
submission_saint_2 = pd.read_csv('saint_transformer_predict_proba_real_final_41.csv')
submission_saint_3 = pd.read_csv('saint_transformer_predict_proba_real_final_1004.csv')

In [84]:
# softmax 함수 정의
def softmax(x):
    e_x = np.exp(x - np.max(x))  # 안정성을 위해 입력에서 최대값을 뺌
    return e_x / e_x.sum(axis=0)

values_1 = submission_saint_1.iloc[:,1:].apply(softmax, axis=1)
values_2 = submission_saint_2.iloc[:,1:].apply(softmax, axis=1)
values_3 = submission_saint_3.iloc[:,1:].apply(softmax, axis=1)

valuees = (values_1+values_2+values_3)/3

In [85]:
final = pd.DataFrame((valuees.values + xtf.values ) /2)

In [86]:
data = (LGBM.values * 0.2  + final.values * 0.8)

In [87]:
submission_saint_1.iloc[:,1:]=data

In [88]:
test_pred_labels_2 = submission_saint_1.iloc[:, 1:].idxmax(axis=1)

In [91]:
subb = pd.read_csv('sample_submission.csv')
subb['SUBCLASS'] = test_pred_labels_2
subb.to_csv('하응.csv', index=False)
subb['SUBCLASS'].value_counts()

SUBCLASS
COAD      543
STES      232
BRCA      230
KIRC      183
KIPAN     136
PRAD      116
LUAD      111
SARC       95
THCA       88
GBMLGG     86
SKCM       85
LGG        80
LIHC       76
HNSC       75
OV         63
UCEC       55
CESC       43
PAAD       40
BLCA       38
TGCT       38
LAML       35
PCPG       34
ACC        21
THYM       18
LUSC       15
DLBC       10
Name: count, dtype: int64

# ML:DL 2:8